In [ ]:
import os
import sys

dir = os.path.abspath('')
while not dir.endswith('ardt'): dir = os.path.dirname(dir)
if not dir in sys.path: sys.path.append(dir)

In [ ]:
import io
import time

import paramiko

from utils.helpers import find_root_dir

from private_keys import GATEWAY_PASSWORD, MYRIAD_PRIVATE_KEY, GITHUB_PRIVATE_KEY

In [ ]:
SCRIPT_TO_RUN = 'run_experiment_test.sh'  # 'run_experiment_1.sh', 'run_experiment_2.sh', etc.

## Connecting to Myriad

In [ ]:
# Private key
k = MYRIAD_PRIVATE_KEY.strip()
padding = len(k) % 4
if padding > 0:
    k += "=" * (4 - padding)
private_key_file = io.StringIO(k)
pkey = paramiko.RSAKey.from_private_key(private_key_file)

# Establish SSH connection to the first server (gateway)
sshgat = paramiko.SSHClient()
sshgat.set_missing_host_key_policy(paramiko.AutoAddPolicy())
sshgat.connect('ssh-gateway.ucl.ac.uk', username='ucabscm', password=GATEWAY_PASSWORD)

# Use this connection to establish a second SSH connection to the second server (myriad)
# Paramiko Transport is used here for nested ssh
transport = sshgat.get_transport()
dest_addr = ('myriad.rc.ucl.ac.uk', 22) # IP and port
local_addr = ('ssh-gateway.ucl.ac.uk', 22) # IP and port
jump = transport.open_channel("direct-tcpip", dest_addr, local_addr)

myriad = paramiko.SSHClient()
myriad.set_missing_host_key_policy(paramiko.AutoAddPolicy())
myriad.connect('myriad.rc.ucl.ac.uk', username='ucabscm', sock=jump, pkey=pkey)

## Set up environment, code, files

In [ ]:
def sftp_upload_dir(sftp, localdir, remotedir):
    os.chdir(os.path.split(localdir)[0])
    parent = os.path.split(localdir)[1]
    for walker in os.walk(parent):
        try:
            sftp.mkdir(os.path.join(remotedir, walker[0]))
        except Exception as e:
            print("Exception:", e)
        for file in walker[2]:
            sftp.put(os.path.join(walker[0], file), os.path.join(remotedir, walker[0], file))

In [ ]:
HOME_DIR = '/home/ucabscm'
ARDT_DIR = find_root_dir()

In [ ]:
# myriad.exec_command('module purge')
# myriad.exec_command('module load default-modules')
# myriad.exec_command('module remove mpi compilers')
# myriad.exec_command('module load python/3.9.10')

In [ ]:
# myriad.exec_command('mkdir ardt-env')
# myriad.exec_command(f'python3 -m venv {HOME_DIR}/ardt-env/ardt')
myriad.exec_command(f'source {HOME_DIR}/ardt-env/ardt/bin/activate.csh')


In [ ]:
myriad.exec_command(f'rm -rf {HOME_DIR}/action-robust-decision-transformer')
myriad.exec_command(f'git clone https://{GITHUB_PRIVATE_KEY}@github.com/afonsosamarques/action-robust-decision-transformer.git')

In [ ]:
myriad.exec_command('pip3 install --upgrade pip')
myriad.exec_command(f'pip3 install -r {HOME_DIR}/action-robust-decision-transformer/requirements.txt')

In [ ]:
myriad.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/eval-outputs')
myriad.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/eval-outputs-pipeline')
myriad.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/eval-outputs-test')
myriad.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/agents')
myriad.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/agents-pipeline')
myriad.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/agents-test')
myriad.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/wandb')
myriad.exec_command(f'mkdir {HOME_DIR}/action-robust-decision-transformer/ardt/wandb-json')

In [ ]:
sftp = myriad.open_sftp()

In [ ]:
sftp.put(f'{ARDT_DIR}/access_tokens.py', f"{HOME_DIR}/action-robust-decision-transformer/ardt/access_tokens.py")

sftp.put(f'{ARDT_DIR}/all-notebooks/clusters-nbs/myriad-run-scripts/{SCRIPT_TO_RUN}', f"{HOME_DIR}/{SCRIPT_TO_RUN}")
myriad.exec_command(f'chmod +x {HOME_DIR}/{SCRIPT_TO_RUN}')

sftp_upload_dir(sftp, f'{ARDT_DIR}/datasets', f"{HOME_DIR}")

In [ ]:
myriad.exec_command(f'cp -r {HOME_DIR}/datasets {HOME_DIR}/action-robust-decision-transformer/ardt')

## Run script and close connections

In [ ]:
myriad.exec_command(f'qsub {HOME_DIR}/{SCRIPT_TO_RUN}')

In [ ]:
time.sleep(60)
sftp.close()
myriad.close()
sshgat.close()